In [9]:
import base64
from io import BytesIO
from rdkit import Chem
from rdkit.Chem import Draw
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from dash import dcc, html, Input, Output, no_update
from jupyter_dash import JupyterDash

In [10]:
df_preds = pd.read_csv('loo_predictions.csv')

In [12]:
fig = px.scatter(df_preds, x="Measured Inhibition (%)", y="Predicted Inhibition (%)",
                    color=df_preds["Predicted Inhibition (%)"] - df_preds["Measured Inhibition (%)"])
# fig = go.Figure(data=[
#     go.Scatter(
#         x=df_preds["Measured Inhibition (%)"],
#         y=df_preds["Predicted Inhibition (%)"],
#         marker=dict(
#         color=df_preds["Predicted Inhibition (%)"] - df_preds["Measured Inhibition (%)"],
#         colorscale='viridis',
#         colorbar=dict(title='Δ Inhibition'),
#         showscale=True
#         ),
#         mode="markers"
#     )
# ])

fig.add_shape(type='line',
              x0=0,
              y0=0,
              x1=100,
              y1=100,
              line=dict(color='black', dash='dash'),
              xref='x',
              yref='y'
              )

# molplot.add_molecules(fig)

# turn off native plotly.js hover effects - make sure to use
# hoverinfo="none" rather than "skip" which also halts events
fig.update_traces(hoverinfo="none", hovertemplate=None)

fig.update_layout(
    title='Leave-One-Out Regression',
    autosize=True,
    width=1200, # size of figure
    height=800,
    xaxis=dict(title="Measured Inhibition (%)"),
    yaxis=dict(title="Predicted Inhibition (%)"),
)
app = JupyterDash(__name__)
app.layout = html.Div([
    dcc.Graph(id="graph-basic-2", figure=fig, clear_on_unhover=True),
    dcc.Tooltip(id="graph-tooltip"),
])

@app.callback(
    Output("graph-tooltip", "show"),
    Output("graph-tooltip", "bbox"),
    Output("graph-tooltip", "children"),
    Input("graph-basic-2", "hoverData"),
)
def display_hover(hoverData):
    if hoverData is None:
        return False, no_update, no_update

    # demo only shows the first point, but other points may also be available
    pt = hoverData["points"][0]
    bbox = pt["bbox"]
    num = pt["pointNumber"]

    df_row = df_preds.iloc[num]
    name = df_row['name']
    smiles = df_row['SMILES']
    pred = df_row['Predicted Inhibition (%)']
    measurement = df_row['Measured Inhibition (%)']

    # if len(desc) > 300:
    #     desc = desc[:100] + '...'

    buffered = BytesIO()
    img = Draw.MolToImage(Chem.MolFromSmiles(smiles))
    img.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue())
    img_str = "data:image/png;base64,{}".format(
        repr(img_str)[2:-1])

    children = [
        html.Div([
            html.Img(src=img_str, style={"width": "100%"}),
            html.H2(f"{name}", style={"color": "darkblue",
                    "font-family": 'Arial'}),
            html.P(f"Prediction (%): {pred:.1f}", style={"color": "black",
                                                         "font-family": 'Arial'}),
            html.P(f"Measurement (%): {measurement:.1f}", style={"color": "black",
                                                                 "font-family": 'Arial'}),
        ], style={'width': '200px', 'white-space': 'normal'})
    ]

    return True, bbox, children

import socket
host = socket.gethostbyname(socket.gethostname())
app.run_server(mode='inline', host=host, height=1000)
# app.run_server(mode='inline', height=1000) # height of dash app inline

/home/rokas/miniconda3/envs/work/lib/python3.7/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

